# Quickstart

This notebook is just to make sure that everything works. Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values. There are two ways to authenticate: using a token or using the Azure CLI (AAD authentication). The Azure CLI is the recommended way.

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

When authenticating using Azure CLI + Azure AD, open a terminal and run the following command: `az login --use-device-code`. This will open a browser window where you can authenticate. After authentication, use Option 1 code to run and you are good to go.

*(You could as well run the following command to get the token: `az account get-access-token --resource https://management.core.windows.net/ --query accessToken -o tsv`. Copy the token and add it to the `.env` file and then use the token to authenticate.)*

> **🔔 Please note that in all following notebooks you will need to set the authentication method accordingly!**

In [7]:
pwd

'/workspaces/azure-openai-in-a-day-workshop/exercises'

In [10]:
%pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached azure_identity-1.6.0-py2.py3-none-any.whl (108 kB)
  Using cached streamlit-1.18.1-py2.py3-none-any.whl (9.6 MB)
  Using cached openai-0.27.1.tar.gz (57 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
  Using cached numpy-1.24.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached pandas-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Using cached matplotlib-3.6.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)
  Using cached plotly-5.12.0-py2.py3-none-any.whl (15.2 MB)
  Using cached scipy-1.10.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
  Using cached scikit_learn-1.2.0-cp39-cp39-manylinux_2_17_x86_64.m

In [5]:
import os
import tiktoken
import openai
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Load environment variables
load_dotenv()

# Option 1 - Use Azure AD authentication with az cli (use az login in terminal)
#default_credential = DefaultAzureCredential()
#token = default_credential.get_token("https://cognitiveservices.azure.com/.default")
#openai.api_type = "azure_ad"
#openai.api_base = os.environ.get("OPENAI_API_BASE")
#openai.api_key = token.token
#openai.api_version = "2022-12-01"

# Option 2 - Using Access Key
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = "2022-12-01"

# Define embedding model and encoding
EMBEDDING_MODEL = 'TextEmbeddingAda002'
COMPLETION_MODEL = 'TextDavinci003'
encoding = tiktoken.get_encoding('cl100k_base')

Test if completitions work:

In [3]:
response = openai.Completion.create(engine="TextDavinci003",
                                    prompt="Knock knock.",
                                    temperature=0)
print(response.choices[0].text)



Who's there?

Boo.

Boo who?



Do it in a streaming fashion:

In [4]:
import sys
for resp in openai.Completion.create(engine='TextDavinci003', prompt='Give me 10 taglines for an ice cream shop', max_tokens=512, stream=True):
    sys.stdout.write(resp.choices[0].text)
    sys.stdout.flush()



1. Cool down with our delicious ice cream!
2. Sweet treats for the whole family!
3. Refresh with our heavenly ice cream!
4. Deliciousness made to perfection!
5. Create sweet memories with our ice cream!
6. Lifes better with a scoop of our cream!
7. Summer never ends at our shop!
8. Our ice cream is the ultimate indulgence!
9. Come get a scoop of happiness!
10. Smile bigger with a scoop of our ice cream!

Test if embeddings work:

In [6]:
e = openai.Embedding.create(input="Hello World!", engine=EMBEDDING_MODEL)["data"][0]["embedding"]
print(e)

[0.0023968014866113663, 0.0003746475849766284, -0.0021421904675662518, -0.025725144892930984, -0.011554940603673458, 0.0009909399086609483, -0.0146354204043746, 0.003445107489824295, 9.989947284338996e-05, -0.027736887335777283, 0.02377627044916153, 0.0050042071379721165, -0.027636300772428513, -0.01019072812050581, 0.007789211813360453, 0.011680674739181995, 0.024995889514684677, -0.014145057648420334, 0.007342856843024492, 0.009662645868957043, -0.007078815717250109, 0.008518468588590622, 0.010310175828635693, 0.005815190263092518, -0.0061169518157839775, 0.0019190128659829497, 0.004884759895503521, -0.01886007748544216, 0.03756927326321602, -0.024115752428770065, 0.016043638810515404, -0.01224019005894661, -0.0031323444563895464, -0.024505527690052986, 0.009769519791007042, -0.011831555515527725, 0.0027504279278218746, -0.012485370971262455, 0.015301809646189213, -0.018558315932750702, 0.008669348433613777, 0.009813526645302773, 0.0048879035748541355, -0.01362954918295145, -0.040788

Test if tokenizer works:

In [7]:
tokens = encoding.encode("Hello world!")
print(tokens)
print(len(tokens))

[9906, 1917, 0]
3


Also let's check if LangChain works:

In [2]:
import os
import openai
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load environment variables
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Create a completion
llm = AzureOpenAI(deployment_name="TextDavinci003")
joke = llm("Tell me a dad joke")
#print(joke)

# Create embeddings
embeddings = OpenAIEmbeddings(model="TextEmbeddingAda002", chunk_size=1)
text = "This is a test document."

print(embeddings)

# Embed a single document
query_result = embeddings.embed_query(text)
print(query_result)

# Embed multiple documents at once
doc_result = embeddings.embed_documents([text, text])

client=<class 'openai.api_resources.embedding.Embedding'> model='TextEmbeddingAda002' document_model_name='TextEmbeddingAda002' query_model_name='TextEmbeddingAda002' embedding_ctx_length=-1 openai_api_key=None chunk_size=1 max_retries=6
[-0.003158408682793379, 0.011094409972429276, -0.0040013170801103115, -0.011747414246201515, -0.0010153218172490597, 0.010781234130263329, -0.010368109680712223, -0.005297331139445305, -0.009881687350571156, -0.02616015262901783, 0.020376399159431458, 0.022575292736291885, -0.007522876374423504, 0.01728462427854538, -0.006003641523420811, 0.01912369765341282, 0.02125595696270466, -0.015645449981093407, 0.007669468875974417, -0.018364081159234047, -0.0006909018848091364, -0.006416766904294491, -0.01098113413900137, 0.017950955778360367, -0.02213551290333271, -0.003076783148571849, 0.01437942124903202, -0.029984891414642334, 0.01852400042116642, -0.007916011847555637, 0.010068260133266449, -0.019456863403320312, -0.003688141703605652, -0.0242411196231842